# Paralelismo en Julia

## 1. Introducción

Julia proporciona distintos mecanismos para paralelizar código. Algunas de las estrategias y desafíos para escribir algoritmos paralelos son los siguientes:  

* Estrategias de paralelismo
     * SIMD
     * Multi-hilo
     * Tareas
     * Multiproceso
         * Memoria compartida
         * Memoria distribuida
     * Programación de GPU

* Desafíos de la computación paralela
     * Orden de ejecución
         * ejecución de fuera de orden de posibilidad
         * acceso y mutación simultáneos
     * Acceso y movimiento de datos
     * Código de acceso y movimiento
     * Adaptación adecuada de la estrategia de paralelismo a las capacidades de su máquina
     * Hacer coincidir adecuadamente la estrategia de paralelismo con el problema en cuestión

## ¿Qué es lo que está sucediendo con nuestras computadoras?

![](https://raw.githubusercontent.com/JuliaComputing/JuliaAcademyData.jl/master/courses/Parallel_Computing/images/40-years-processor-trend.png)


## Lo difícil de la computación paralela
   * No pensamos en paralelo
   * Aprendemos a escribir y razonar sobre programas en serie.
   * El deseo de paralelismo a menudo surge _después_ de haber escrito su algoritmo (¡y lo encontró demasiado lento!)

## Resumen:
   * Las arquitecturas computacionales actuales nos empujan hacia la programación paralela para un rendimiento máximo, ¡incluso si no estamos en un clúster!
   * Pero es difícil diseñar buenos algoritmos paralelos
   * Es difícil de expresar y razonar sobre esos algoritmos.

## 2. SIMD: El paralelismo que puede (a veces) suceder automáticamente

SIMD: Instrucción única, datos múltiples (Single Instruction Multiple Data)

**Nota:** También llamado confusamente vectorización

### Arquitectura

En lugar de calcular cuatro sumas secuencialmente:

\begin{align}
x_1 + y_1 &\rightarrow z_1 \\
x_2 + y_2 &\rightarrow z_2 \\
x_3 + y_3 &\rightarrow z_3 \\
x_4 + y_4 &\rightarrow z_4
\end{align}

Procesadores modernos tienen unidades de procesamiento vectorial que pueden hacer lo anterior a la vez:

$$
\left(\begin{array}{cc}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}\right)
+
\left(\begin{array}{cc}
y_1 \\
y_2 \\
y_3 \\
y_4
\end{array}\right)
\rightarrow
\left(\begin{array}{cc}
z_1 \\
z_2 \\
z_3 \\
z_4
\end{array}\right)
$$

### ¿Cómo se logra?

In [ ]:
using BenchmarkTools

In [ ]:
A = rand(100_000)
function simplesum(A)
    result = zero(eltype(A))
    for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end

simplesum(A)

Como muchos lenguajes de programación modernos, Julia utiliza la verificación de límites ([_boundchecking_](https://docs.julialang.org/en/v1/devdocs/boundscheck/)) para garantizar la seguridad del programa al acceder a arreglos.
En bucles internos u otras situaciones críticas de rendimiento, es posible que se desee omitir estas comprobaciones de límites para mejorar el rendimiento en tiempo de ejecución.

En consecuencia, Julia incluye la macro `@inbounds(...)` para decirle al compilador que omita dichas comprobaciones de límites dentro del bloque dado.

In [ ]:
@btime simplesum($A)

¿ese tiempo es bueno?

In [ ]:
@btime sum($A)

Diseñamos una función más lenta que la suma prediseñada `sum()`, ¡y también estamos obteniendo una respuesta diferente! Veamos qué sucede con un flotante de 32 bits en lugar de uno de 64 bits. Cada elemento tiene la mitad del número de bits, por lo que también permite duplicar la longitud (para que el número total de bits procesados permanezca constante).

In [ ]:
A32 = rand(Float32, length(A)*2)
@btime simplesum($A32)
@btime sum($A32)

¡Eso es aun peor! ¿Que está pasando aqui? 
Estamos viendo múltiples diferencias en el desempeño: ¿quizás la suma incorporada de Julia está usando algún paralelismo?

Intentemos usar SIMD nosotros mismos:

In [ ]:
function simdsum(A)
    result = zero(eltype(A))
    @simd for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end
@btime simdsum($A)
@btime simdsum($A32)

¿Qué hizo y por qué no siempre usamos (usa Julia pues) `@simd` para cada bucle **for** automáticamente?

Veamos los resultados:

In [ ]:
simplesum(A), simdsum(A), sum(A)

In [ ]:
simplesum(A32), simdsum(A32), sum(A32)

¿Por qué no son iguales?

Sin `@simd`, Julia está haciendo _exactamente_ lo que le dijimos que hiciera: está tomando cada elemento de nuestro arreglo y lo agrega a una gran pila secuencialmente. Nuestra respuesta es más pequeña de lo que la "suma" incorporada de Julia cree que es: eso es porque, como la pila se hace más grande, comenzamos a perder las partes inferiores de cada elemento que estamos sumando, ¡y esas pequeñas pérdidas comienzan a acumularse!

La macro `@simd` le dice a Julia que puede reorganizar las adiciones de punto flotante -
incluso si cambiara la respuesta. Dependiendo de su CPU, esto puede llevar a 2x o 4x
o incluso un paralelismo 8x. Básicamente, Julia está calculando sumas independientes para
los índices pares y los índices impares simultáneamente:

\begin{align}
odds &\leftarrow 0 \\
evens &\leftarrow 0 \\
\text{loop}&\ \text{odd}\ i: \\
    &\left(\begin{array}{cc}
odds \\
evens
\end{array}\right)
\leftarrow
\left(\begin{array}{cc}
odds \\
evens
\end{array}\right)
+
\left(\begin{array}{cc}
x_{i} \\
x_{i+1}
\end{array}\right) \\
total &\leftarrow evens + odds
\end{align}


En muchos casos, Julia puede y sabe que un bucle for puede ser vectorizado (SIMD-ed) y aprovechará esto por defecto.

In [ ]:
B = rand(1:10, 100_000)
@btime simplesum($B)
@btime sum($B)
B32 = rand(Int32(1):Int32(10), length(B)*2)
@btime simplesum($B32)
@btime simdsum($B32)

¿Cómo inspeccionamos que se está vectorizando?

In [ ]:
@code_llvm simdsum(A32)

Entonces, ¿cuáles son los desafíos?:

- El mayor obstáculo es que tienes que convencer a Julia y LLVM de que puede usar instrucciones SIMD para tu algoritmo dado. Eso no siempre es posible.
- Hay muchas limitaciones de lo que se puede y no se puede vectorizar
- Es necesario pensar en las consecuencias de reordenar su algoritmo

## Resumen

SIMD:
- Explota el paralelismo integrado en un procesador
- Ideal para bucles internos pequeños (y estrechos)
- A menudo ocurre automáticamente si tienes cuidado
    - Sigue las [mejores prácticas de rendimiento](https://docs.julialang.org/en/v1/manual/performance-tips/)
    - Usa `@inbounds` para cualquier acceso a un arreglo
    - Evita ramas o llamadas a funciones
- Dependiendo del procesador y los tipos involucrados, puede producir ganancias de 2-16x con una sobrecarga extraordinariamente pequeña
